In [15]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=0, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)

In [29]:
indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

In [30]:
Mindexed_circuit, Mresolver, Mindex_to_symbols = iid_inserter.place_identities(indexed_circuit,symbol_to_value=resolver)
Mcircuit = vqe_handler.give_unitary(Mindexed_circuit, Mresolver)

In [31]:
model = QNN(symbols=list(Mresolver.keys()), observable=vqe_handler.observable)
print(model(circuit), model(Mcircuit))

tf.Tensor([2.522305], shape=(1,), dtype=float32) tf.Tensor([-3.9750285], shape=(1,), dtype=float32)


In [32]:
OMenergy, OMresolver, OMhistory = vqe_handler.vqe(Mindexed_circuit, symbols_to_values=Mresolver)
OMcircuit = vqe_handler.give_unitary(Mindexed_circuit, OMresolver)

In [33]:
print(model(Mcircuit), model(OMcircuit))

tf.Tensor([-3.9750285], shape=(1,), dtype=float32) tf.Tensor([-3.999978], shape=(1,), dtype=float32)


In [34]:
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.reduce_circuit(Mindexed_circuit, OMresolver, Mindex_to_symbols)

connections
{'0': ['rx'], '1': ['rz', 'rx', 'rz', 'rx']}


places_gates:
{'0': [0], '1': [1, 2, 3, 4]}


******
{'th_0': -18.849665}
[4, 3, 5, 3, 5]
{'0': ['th_0'], '1': []}
******
{'th_0': -18.849665, 'th_1': 0.034886662, 'th_2': -0.08674769, 'th_3': -15.742888}
[4, -1, 5, 3, 5]
{'0': ['th_0'], '1': ['th_1', 'th_2', 'th_3']}
connections
{'0': ['rx'], '1': ['rx', 'rz', 'rx']}


places_gates:
{'0': [0], '1': [1, 2, 3]}


******
{'th_0': -18.849665}
[4, 5, 3, 5]
{'0': ['th_0'], '1': []}
******
{'th_0': -18.849665, 'th_1': 0.034886662, 'th_2': -0.08674769, 'th_3': -15.742888}
[4, 5, 3, 5]
{'0': ['th_0'], '1': ['th_1', 'th_2', 'th_3']}
connections
{'0': ['rx'], '1': ['rx', 'rz', 'rx']}


places_gates:
{'0': [0], '1': [1, 2, 3]}


******
{'th_0': -18.849665}
[4, 5, 3, 5]
{'0': ['th_0'], '1': []}
******
{'th_0': -18.849665, 'th_1': 0.034886662, 'th_2': -0.08674769, 'th_3': -15.742888}
[4, 5, 3, 5]
{'0': ['th_0'], '1': ['th_1', 'th_2', 'th_3']}
connections
{'0': ['rx'], '1': ['rx', 'rz', 'rx'

In [35]:
Scircuit = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
print(model(Scircuit),OMenergy)

tf.Tensor([-3.9999774], shape=(1,), dtype=float32) tf.Tensor(-3.9999776, shape=(), dtype=float32)


In [36]:
Scircuit

(0, 0): ───Rx(2π)──────────────────────────────────

(0, 1): ───Rx(0.011π)───Rz(-0.028π)───Rx(-1.01π)───

In [37]:
OMcircuit

(0, 0): ───Rx(2π)────────────────────────────────────────────────

(0, 1): ───Rz(-0.032π)───Rx(0.011π)───Rz(-0.028π)───Rx(-1.01π)───